In [ ]:
import ciao_contrib.runtool as rt
import astropy.io.fits as fits
import numpy as np
import pandas as pd
import os

In [ ]:
acis_src = pd.read_csv('acis_sources.csv')
source_no = acis_src['source_no.']
obsid = acis_src['obsid']
spec_info = open('spec_info.csv','a')
spec_info.write('source_no.,obsid,grouping\n')
spec_info.close()

In [ ]:
grp = 20
for i in range(len(obsid)):
    repro_path = f'{source_no[i]}/{obsid[i]}/repro/'
    os.system(f'ls {repro_path}*repro_evt2.fits > {repro_path}repro_evt.txt')
    evt_filename = open(f'{repro_path}repro_evt.txt').readlines()[0][len(repro_path):-1] #reprocessed event file nameevt_filename = ''
    rt.specextract.punlearn()
    rt.specextract.infile = f'{repro_path}{evt_filename}[sky=region({repro_path}source_region.reg)]'
    rt.specextract.bkgfile = f'{repro_path}{evt_filename}[sky=region({repro_path}background_region.reg)]'
    rt.specextract.outroot = f'{repro_path}spectrum/spec_'
    rt.specextract.grouptype = 'NUM_CTS'
    rt.specextract.clobber = 'YES'
    rt.specextract.weight = 'no'
    rt.specextract.correctpsf = 'yes'
    rt.specextract.energy_wmap = '300:11000'
    try :
        rt.specextract(binspec = grp)
        n_bins = np.max(fits.open(f'{source_no[i]}/{obsid[i]}/repro/spectrum/spec_grp.pi')[1].data['GRP_NUM'])
        min_bins = 15
        '''
            If no of data points is less than {min_bins} then grouping no. will be reduced by 5 starting from 25 until no. of data points reach
            {min_bins}. But grouping can be minimum at 15, if even with that data points count doesn't reach {min_bins} then grouping will be
            done such that there is atleast one count in each bin. In that case c-stat should be used for modelling instead of chi_sq methond.
            {min_bins} by default is chosen to be 25.
        '''
        while n_bins < min_bins and grp > 15:
            grp = grp - 5
            rt.specextract(binspec = grp)
            n_bins = np.max(fits.open(f'{source_no[i]}/{obsid[i]}/repro/spectrum/spec_grp.pi')[1].data['GRP_NUM'])

        if n_bins < min_bins :
            grp = 1
            rt.specextract(binspec = grp)
        
        n_bins = np.max(fits.open(f'{source_no[i]}/{obsid[i]}/repro/spectrum/spec_grp.pi')[1].data['GRP_NUM'])
        print(f'{source_no[i]}\t{obsid[i]}\tNo. of bins = {n_bins} with grouping at {grp}')
        spec_info = open('spec_info.csv','a')
        spec_info.write(f'{source_no[i]},{obsid[i]},{grp}\n')
        spec_info.close()
        
    except :
        prob_source = open('problematic_source.txt','a')
        prob_source.write(f'{source_no[i]}\t{obsid[i]}\n')
        prob_source.close()
        print(f'{source_no[i]}\t{obsid[i]}\tproblems (spectrum not extracted)')
        continue